## ALL INSTALLS

In [1]:



!pip install langchain
!pip install unstructured
!pip install pdf2image
!pip install chromadb
!pip install tiktoken


!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding faiss-cpu

!pip install faiss-cpu

!pip install -q transformers einops accelerate langchain bitsandbytes


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 13.1 MB/s eta 0:00:00
 

# ALL IMPORTS

In [2]:


from langchain.document_loaders import DirectoryLoader,TextLoader,PyPDFLoader 
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma,FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
# InstructorEmbedding 
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain import HuggingFacePipeline




from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

import textwrap
import torch
import pickle
import faiss
import os



/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# PDF Data Loader

In [4]:


loader_pdf = DirectoryLoader('/content/drive/MyDrive/data_QA', glob="**/*.pdf")
# loader_text = TextLoader('/content/drive/MyDrive/data/WhatsApp Chat with EAGLE🦅.txt')
docs = loader_pdf.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
text_splitter = RecursiveCharacterTextSplitter(
                                               chunk_size=1000, 
                                               chunk_overlap=200)

texts = text_splitter.split_documents(docs)



# Question List

In [6]:
query_ls  = [
    "When did the GARDASIL 9 recommendations change?",
    "What were the past 3 recommendation changes for GARDASIL 9?",
    "Is GARDASIL 9 recommended for Adults?",
    "Does the ACIP recommend one dose GARDASIL 9?"
]

# Embeddings for OUR Documents 

In [7]:
def store_embeddings(docs, embeddings, sotre_name, path):
    
    vectorStore = FAISS.from_documents(docs, embeddings)

    with open(f"{path}/faiss_{sotre_name}.pkl", "wb") as f:
        pickle.dump(vectorStore, f)

def load_embeddings(sotre_name, path):
    with open(f"{path}/faiss_{sotre_name}.pkl", "rb") as f:
        VectorStore = pickle.load(f)
    return VectorStore

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
#                                                       model_kwargs={"device": "cuda"})







In [8]:
Embedding_store_path = f"/content/drive/MyDrive/Embedding_store"

In [11]:
# store_embeddings(texts, 
#                  instructor_embeddings, 
#                  sotre_name='instructEmbeddings', 
#                  path=Embedding_store_path)

In [9]:
db_instructEmbedd = load_embeddings(sotre_name='instructEmbeddings', 
                                    path=Embedding_store_path)

In [10]:
# db_instructEmbedd = FAISS.from_documents(texts, instructor_embeddings)

retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 3})

In [11]:
retriever.search_type

'similarity'

# Get Context Paragraphs for each Question

In [12]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [13]:
for query in query_ls:
  print(query)
  result = retriever.get_relevant_documents(query)
  for ans in result:
    print(ans,)
    print("###"*8)
  print("---"*8)

When did the GARDASIL 9 recommendations change?
page_content='What is added by this report?\n\nAfter reviewing new evidence, CDC updated HPV vaccination recommendations for U.S. adults.\n\nWhat are the implications for public health practice?\n\nRoutine recommendations for HPV vaccination of adolescents have not changed. Catch-up HPV vaccination is now recom- mended for all persons through age 26 years. For adults aged 27 through 45 years, public health benefit of HPV vaccination in this age range is minimal; shared clinical decision-making is recommended because some persons who are not adequately vaccinated might benefit.\n\nvaccination should be delayed until after pregnancy; however, pregnancy testing is not needed before vaccination. Persons who are breastfeeding or lactating can receive HPV vaccine. Recommendations regarding HPV vaccination during pregnancy or lactation have not changed (1).\n\nFuture Research and Monitoring Priorities' metadata={'source': '/content/drive/MyDrive

 # Open Source Model For QA Task 

In [24]:
model_id = "distilgpt2"

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)

In [26]:
pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.2,
    top_p=0.95,
    repetition_penalty=1.15,
)

llm_os = HuggingFacePipeline(pipeline=pipe) 

In [21]:
# create the chain to answer questions 
qa_chain_instrucEmbed = RetrievalQA.from_chain_type(llm_os, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)


In [22]:


def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [27]:
for query in query_ls:
  print('-------------------Instructor Embeddings------------------\n')
  try :
    print(query)
    llm_response = qa_chain_instrucEmbed(query)
    process_llm_response(llm_response)
  except Exception as e :
    #print(e)
    print("Out of Scope")



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------------------Instructor Embeddings------------------

When did the GARDASIL 9 recommendations change?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 The recommendation changes were made based upon research findings from previous studies published prior year
which indicated no significant difference in efficacy between these two doses compared against other oral
contraceptives such Aspergillus cerevisiae strains [see Table 4]. However there has been considerable debate
about whether any differences exist within each dose group when given together due either to their different
pharmacokinetics relative potency versus placebo vs fluoxetine alone administered orally over time rather than
using single administration agents like aspirin combined.[3] There may be several reasons why certain
formulations differ depending how well they interact but most importantly it's important we understand what
works best while minimizing side effects associated both directly with individual users[4][5], thus making
informed decisions concerning dosage choices among various types thereof including combinations where
possible,[6],[7]). Therefore I would

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Out of Scope
-------------------Instructor Embeddings------------------

Is GARDASIL 9 recommended for Adults?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Yes! The Garda's Guide To Preventing Cancer has been published since 1995 which provides information about
how bestto prevent cervical cancers including prevention strategies such Asperger syndrome, autism spectrum
disorders, postmenopausal women, adolescents diagnosed early during pregnancy/pregnancy before birth, preterm
infants born between ages 2–6 months, young people living alone without parental consent under certain
circumstances, pregnant teenagers receiving caregiving services outside their home country(s); youth whose
parents were sexually active but did NOT vaccinate them against any disease other than measles when they
became aware it occurred within 3 weeks of being admitted into hospital due only TO THE FACT OF A CHILD WHO
WAS STILL IN HER MOTHER'S HOME AND HAS NEVER BEEN HELD WITH ANY OTHER PERSON OR ENTITY FOR MORE THAN THREE
YEAR SIX MONTHLY AFTER HE STARTED PUTTING HIS PARENTS INTO AN ADULT CARE CENTER BEFORE THEY WERE ABLETO INFANT
BECAUSE IT IS PREVIOUS THAT SOMEO

In [ ]:
query = "When did the GARDASIL 9 recommendations change?"
index.query(query)

In [ ]:
query = "When did the GARDASIL 9 recommendations change?"
index.query_with_sources(query)

In [ ]:
for query in query_ls:
  print(index.query_with_sources(query))





# QA Model with Context

In [39]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2-covid"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [40]:
res

{'score': 0.03949260711669922,
 'start': 59,
 'end': 132,
 'answer': 'gives freedom to the user and let people easily switch between frameworks'}

In [41]:
for query in query_ls:
  print(f"QUERY :: {query}")
  result = retriever.get_relevant_documents(query)
  for context in result:
    QA_input = {
      'question': query,
      'context': context.page_content }
    print("----"*10)
    answer  = nlp(QA_input)
    print(answer)
    print("\n\n")
    print("****"*10)

QUERY :: When did the GARDASIL 9 recommendations change?
----------------------------------------
{'score': 3.7271659039106453e-06, 'start': 747, 'end': 835, 'answer': 'Recommendations regarding HPV vaccination during pregnancy or lactation have not changed'}



****************************************
----------------------------------------
{'score': 0.00949524249881506, 'start': 11, 'end': 24, 'answer': 'February 2015'}



****************************************
----------------------------------------
{'score': 0.1538630723953247, 'start': 635, 'end': 652, 'answer': 'December 10, 2014'}



****************************************
QUERY :: What were the past 3 recommendation changes for GARDASIL 9?
----------------------------------------
{'score': 3.6329885233499226e-07, 'start': 343, 'end': 393, 'answer': 'human papillomavirus 9-valent vaccine, recombinant'}



****************************************
----------------------------------------
{'score': 4.305237055746147e-09, 'star

In [34]:
type(result[0].text)

langchain.schema.Document

In [36]:
result[0].page_content 

'During its February 2015 meeting, the Advisory Committee on Immunization Practices (ACIP) recommended 9-valent human papillomavirus (HPV) vaccine (9vHPV) (Gardasil 9, Merck and Co., Inc.) as one of three HPV vaccines that can be used for routine vaccination (Table 1). HPV vaccine is recom- mended for routine vaccination at age 11 or 12 years (1). ACIP also recommends vaccination for females aged 13 through 26 years and males aged 13 through 21 years not vaccinated previ- ously. Vaccination is also recommended through age 26 years for men who have sex with men and for immunocompromised persons (including those with HIV infection) if not vaccinated previously (1). 9vHPV is a noninfectious, virus-like particle (VLP) vaccine. Similar to quadrivalent HPV vaccine (4vHPV), 9vHPV contains HPV 6, 11, 16, and 18 VLPs. In addition, 9vHPV contains HPV 31, 33, 45, 52, and 58 VLPs (2). 9vHPV was approved by the Food and Drug Administration (FDA) on December 10, 2014, for use in females aged 9'

In [ ]:
len(texts)

In [ ]:
retriever.search_type

In [ ]:
retriever.search_kwargs

In [ ]:
docs = retriever.get_relevant_documents("Who are the authors of GPT4All report?")

In [ ]:
docs[0]

In [ ]:
for query in query_ls:
  print('-------------------OpenAI Embeddings------------------')
  llm_response = qa_chain_openai(query)
  process_llm_response(llm_response)
  print('\n\n\n')

In [ ]:
!nvidia-smi